In [3]:
import json, re, ast

In [9]:
FASHION_METAFILE="../data/fashion_prefab_metadata_all.json"
FURNITURE_METAFILE="../data/furniture_prefab_metadata_all.json"

with open(FASHION_METAFILE) as f:
    fash_meta = json.load(f)

with open(FURNITURE_METAFILE) as f:
    fur_meta = json.load(f)

len(fash_meta) + len(fur_meta)

345

In [61]:
name2id_fash = dict()
for id, name in enumerate(fash_meta):
    name2id_fash[name] = id

name2id_fur = dict()
for id, name in enumerate(fur_meta):
    name2id_fur[name] = id

id2name_fash = dict()
for id, name in enumerate(fash_meta):
    id2name_fash[id] = name

id2name_fur = dict()
for id, name in enumerate(fur_meta):
    id2name_fur[id] = name

print(name2id_fash['WomensCollection/Prefabs/Rearranged/leather_jacket_1'])
print(id2name_fash[116])
print(fash_meta[id2name_fash[116]])
print(fur_meta[id2name_fur[16]])

116
WomensCollection/Prefabs/Rearranged/leather_jacket_1
{'assetType': 'jacket_hanging', 'customerReview': 3.5, 'availableSizes': ['XL', 'M'], 'color': 'brown', 'pattern': 'plain', 'brand': 'Coats & More', 'sleeveLength': 'long', 'type': 'jacket', 'price': 199.99, 'size': 'XL'}
{'brand': '212 Local', 'color': 'black and white', 'customerRating': 3.7, 'materials': 'leather', 'price': '$499', 'type': 'Chair'}


In [64]:
def insert_attributes(line):
    pos = 0
    idx = line.find("<@", pos)
    while idx != -1:
        # get absolute object ID
        abs_id = line[idx+2:idx+6]
        # get object type
        meta = abs_id[0]
        abs_id = int(abs_id[1:])
        obj_brand = fash_meta[id2name_fash[abs_id]]['brand'] if meta=='1' else fur_meta[id2name_fur[abs_id]]['brand']
        obj_price = fash_meta[id2name_fash[abs_id]]['price'] if meta=='1' else fur_meta[id2name_fur[abs_id]]['price']
        # append object type to line
        line = line[:idx+7] + obj_brand + str(obj_price) + line[idx+7:]
        # update pos and idx
        pos = idx+4
        idx = line.find("<@", pos)
    return line

In [65]:
with open("../models/bart_coref/data_object_special/simmc2_dials_dstc10_train_predict.txt", encoding="utf-8") as f:
    for line in f.read().splitlines():
        if (len(line) > 0 and not line.isspace()):
            line_boxes = [ast.literal_eval(position.replace('(', '').replace(')', '')) for position in re.findall(r"\[\([^\)]+\)\]", line)]
            line = re.sub(r"\[\([^\)]*\)\]", "", line)
            l = insert_attributes(line)
            print("Line before insertion:", line)
            print("Line after insertion:", l)
            break

Line before insertion: User : Hi, do you have any jackets today? <SOO><NOCOREF><OBJ><59><@1116><OBJ><80><@1127><OBJ><55><@1050><OBJ><67><@1132><OBJ><68><@1227><OBJ><70><@1235><OBJ><82><@1052><OBJ><79><@1283><OBJ><69><@1225><OBJ><72><@1137><OBJ><71><@1233><OBJ><73><@1109><OBJ><61><@1013><OBJ><62><@1015><OBJ><63><@1015><OBJ><58><@1018><OBJ><74><@1283><OBJ><75><@1166><OBJ><64><@1053><OBJ><76><@1067><EOO> => Belief State : 
Line after insertion: User : Hi, do you have any jackets today? <SOO><NOCOREF><OBJ><59><@1116>Coats & More199.99<OBJ><80><@1127>The Vegan Baker99.99<OBJ><55><@1050>Global Voyager199.99<OBJ><67><@1132>Pedals & Gears54.99<OBJ><68><@1227>212 Local144.99<OBJ><70><@1235>The Vegan Baker149.99<OBJ><82><@1052>Pedals & Gears44.99<OBJ><79><@1283>Uptown Gallery224.99<OBJ><69><@1225>Uptown Gallery94.99<OBJ><72><@1137>HairDo244.99<OBJ><71><@1233>Downtown Consignment134.99<OBJ><73><@1109>Cats Are Great94.99<OBJ><61><@1013>Downtown Consignment174.99<OBJ><62><@1015>Modern Arts184.99<OB